<a href="https://colab.research.google.com/github/fefespn/HoloLens_Speach_Face_Person_Recognition/blob/master/BF_For_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install bloom-filter
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from bloom_filter import BloomFilter
import numpy as np
import matplotlib.pyplot as plt


Import Fashion_MNIST. 70,000 grayscale images in 10 categories.
We look at the data set in this way:
- all the univirse  is the 70,000 images
- our positive keys is images with label 0. There is 1,000 images like that.


In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images_, train_labels), (test_images_, test_labels) = fashion_mnist.load_data()

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images_ / 255.0

test_images = test_images_ / 255.0

In [0]:
#bloom = BloomFilter(max_elements=6000, error_rate=0.05)
bloom = BloomFilter()
tshirt_counter = 0

for i in range(len(train_labels)):
    if(train_labels[i] == 0):
        img_encode = np.ravel(train_images_[i]).tolist()
        bloom.add(img_encode)
        tshirt_counter = tshirt_counter + 1
        
print("T-Shirt count = ",tshirt_counter)
c = 0
for i in range(len(train_labels)):
    img_encode = np.ravel(train_images_[i]).tolist()
    if(img_encode in bloom):
        c = c+1
print("bloom filter T-Shirt count=",c)
print("bloom filter Train accuracy=",(c-tshirt_counter)/50000,'\n\n')




T-Shirt count =  6000
bloom filter T-Shirt count= 7281
bloom filter Train accuracy= 0.02562 




In [0]:

#bloom1 = BloomFilter()
tshirt_counter = 0
for i in range(len(test_labels)):
    if(test_labels[i] == 0):
        #img_encode = np.ravel(test_images_[i]).tolist()
        #bloom1.add(img_encode)
        tshirt_counter = tshirt_counter + 1
        
print("T-Shirt count = ",tshirt_counter)
c = 0
for i in range(len(test_labels)):
    img_encode = np.ravel(test_images_[i]).tolist()
    if(img_encode in bloom):
        c = c+1
print("bloom filter T-Shirt count=",c)
print("bloom filter accuracy=",(c-tshirt_counter)/9000)

T-Shirt count =  1000
bloom filter T-Shirt count= 241
bloom filter accuracy= -0.08433333333333333


In [0]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [7]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/5
60000/60000 [==============================] - 6s 106us/sample - loss: 0.4940 - acc: 0.8244
Epoch 2/5
60000/60000 [==============================] - 5s 90us/sample - loss: 0.3749 - acc: 0.8645
Epoch 3/5
60000/60000 [==============================] - 5s 90us/sample - loss: 0.3388 - acc: 0.8755
Epoch 4/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.3159 - acc: 0.8843
Epoch 5/5
60000/60000 [==============================] - 6s 98us/sample - loss: 0.2962 - acc: 0.8904


In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('\nTest accuracy:', test_acc)

10000/10000 [==============================] - 1s 52us/sample - loss: 0.3615 - acc: 0.8634

Test accuracy: 0.8634


In [0]:
predictions = model.predict(test_images)

In [10]:
np.argmax(predictions[0])

9